In [ ]:
import time
import copy
import pathlib

import tqdm
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd

import shapely.geometry

import sys
sys.path.insert(0, r'C:\Users\Wei Zhou\Documents\zhouwei file\Github-Project\VeTool-Processing-OSM')
from osm_process_tool.network.diagnosis import print_graph_info
from code_utils.utils_basic import PROJECT_PATH, PROJECTED_CRS

# 0. Extract OSM walking network using `osmium` tool

Extract walking network through `osmium` tool

```PowerShell
cd %YOUR_DATA_PATH%
```

- Extract OSM data for a specific date

``` PowerShell
osmium time-filter ^
    -o "china-20221231.osm.pbf" ^
    "INPUT.osh.pbf" ^
    2022-12-31T00:00:00Z
```

- Extract OSM data for a specific region

```PowerShell
osmium extract ^
    -p "chongqing.poly" ^
    -o "chongqing-20221231.osm.pbf" ^
    -s complete_ways ^
    "china-20221231.osm.pbf"
```

- Extract walking network

```PowerShell
osmium tags-filter ^
    chongqing-20221231.osm.pbf ^
    w/highway ^
    w/area!=yes ^
    w/access!=private ^
    w/highway!=abandoned,bus_guideway,construction,cycleway,motor,no,planned,platform,proposed,raceway,razed,rest_area,services ^
    w/foot!=no ^
    w/service!=private ^
    w/sidewalk!=separate ^
    w/sidewalk:both!=separate ^
    w/sidewalk:left!=separate ^
    w/sidewalk:right!=separate ^
    -o chongqing-20221231-walking.osm
```

# 1. Load data

#### (1) Load boundary

In [ ]:
import sys
sys.path.insert(0, r'C:\Users\Wei Zhou\Documents\zhouwei file\Github-Project\VeTool-sgdatabase_utils')
import sgdatabase_utils.load_data.load_database_datasets as loaddb

# load boundary
boundary = loaddb.load_zone_geometry('boundary', year=2019, with_sea=True)
print('CRS of boundary:', boundary.crs)
boundary = boundary.geometry[0]

#### Load walking network by using `OSMnx`

In [ ]:
import osmnx
if 'crossing' not in osmnx.settings.useful_tags_way:
    osmnx.settings.useful_tags_way.append('crossing')
print(osmnx.settings.useful_tags_way)

In [ ]:
# Extract walking network from OSM data
# Require about 2 min
filepath = PROJECT_PATH / 'data/osm map/network_walking.osm'
network = ox.graph.graph_from_xml(
    filepath, 
    bidirectional = True, 
    simplify = True, 
    retain_all = True, 
    encoding = 'utf-8')
# Return type: MultiDiGraph
# Node attributes: ['node', 'y', 'x', 'highway', 'ref', 'junction', 'railway']
# Edge attributes: [
#   'source', 'target', 'osmid', 'highway', 'lanes', 'maxspeed', 'name',
#   'oneway', 'ref', 'reversed', 'length', 'bridge', 'geometry', 'length_m',
#   'tunnel', 'access', 'service', 'junction', 'width', 'crossing',
#   'est_width', 'area']
# node attributes ('x', 'y') indicating the geographic coordinates (i.e., longitude and latitude)
# edge attributes: ('geometry'), some edges may miss geometry attribute
print_graph_info(network)

# 2. Preprocess the network topology

In [ ]:
from osm_process_tool.network.osm_network_preprocess import (
    remove_nodes_outside_boundary,
    reproject_network_geometry,
    collapse_multidigraph_to_graph,
    process_isolated_nodes,
    graph_to_geodataframe)

#### (1) Clip nodes by boundary

Extract graph nodes within specified boundary.

- Consider the nodes' coordinates (usually specify by their attributes `x`, `y`) from the OSM data are usually in geographic coordinates (i.e., latitude and longitude, WGS84 system, EPSG:4326). Their coordinates are reprojected to a projected CRS (specify by parameter `projected_crs`) before clipping the nodes by the boundary.

- This function will NOT change the nodes' coordinates, but only remove the nodes outside the boundary.

In [ ]:
# Clip nodes by boundary
network = remove_nodes_outside_boundary(
    network,
    projected_crs = PROJECTED_CRS,
    boundary = boundary,
    node_attr_x = 'x',
    node_attr_y = 'y',
    source_crs = 'EPSG:4326',)
print('Clip network nodes by boundary')
print_graph_info(network)

#### (2) Reproject network

1. Reproject nodes and edges to a projected CRS (argument `projected_crs`)
2. Add straight-line length for edges missing geometry (argument `edge_attr_geom`) attribute
3. Compute edge length in meters and add associated attribute (argument `edge_attr_len`)


In [ ]:
# Reproject the network
network = reproject_network_geometry(
    network,
    source_crs = 'EPSG:4326',
    projected_crs = PROJECTED_CRS,
    edge_attr_geom = 'geometry',
    edge_attr_len = 'length_m',
    edge_non_geom_add = True,     # Add straight-line for edges missing geometry
    node_attr_x = "x",
    node_attr_y = "y",
    node_attr_proj_x = "proj_x",
    node_attr_proj_y = "proj_y",
    node_non_geom_remove = True,  # Remove node missing valid coordinates)
)
print('Reproject network')
print_graph_info(network)

#### (4) Collapse to undirected graph

1. Using the minimum length for edges with the same start and end node
2. Remove self-loops edges (same start and end node)

In [ ]:
network = collapse_multidigraph_to_graph(
    network,
    weight = "length_m"
)
print('Collapse to undirected graph')
print_graph_info(network)

#### (5) Connect isolated nodes to network

1. Connect isolated nodes to the network nodes based on a threshold distance (argument `threshold`).


In [ ]:
# Connect isolated nodes to network
network = process_isolated_nodes(
    network,
    threshold = 100.,         # threshold distance for isolated nodes
    node_attr_x = 'proj_x',
    node_attr_y = 'proj_y',
    edge_attr_geom = 'geometry',
    edge_attr_len = 'length_m'
)
print('Connect isolated nodes to network')
print_graph_info(network)

In [ ]:
# update node ids:
mapper = {n: f"W_{n}" for n in network.nodes()}
network = nx.relabel_nodes(network, mapper)
print_graph_info(network)

# 3. Preprocess the network node and edge attributes

#### Clear edge attribute values

In [ ]:
def get_edge_attribute_value(attr_value):

    if isinstance(attr_value, list):
        attr_value = ','.join(attr_value)
    elif isinstance(attr_value, str):
        attr_value = attr_value
    else:
        print('Error for', attr_value)
        attr_value = None

    return attr_value

In [ ]:
# 'crossing'
for u, v, attrs in tqdm.tqdm(network.edges(data=True), desc='Update edge attributes'):
    # update 'highway'
    attr_li = list(attrs.keys())

    attr_name = 'highway'
    if attr_name in attr_li:
        attr_value = get_edge_attribute_value(attrs[attr_name])
        attrs[attr_name] = attr_value.split(',')[0]
    else:
        attrs[attr_name] = 'unknown'

    attr_name = 'crossing'
    if attr_name in attr_li:
        attr_crossing = get_edge_attribute_value(attrs[attr_name])
        attrs[attr_name] = 'signal' if ('signal' in attr_crossing) else 'crossing'
    else:
        attrs[attr_name] = 'unknown'

    for attr_name in ['bridge', 'tunnel']:
        if attr_name in attr_li:
            attr_value = get_edge_attribute_value(attrs[attr_name])
            attrs[attr_name] = 'no' if ('no' in attr_value) else 'yes'
        else:
            attrs[attr_name] = 'unknown'

#### (2) Remove unnecessary node and edge attributes

In [ ]:
from code_utils.network_process.network_preprocess import (
    remove_node_edge_attrs, remove_edge_by_attr_value)

In [ ]:
# remove unnecessary attributes
remove_node_attr = [
    'ref', 'junction', 'railway']
remove_edge_attr = [
    'osmid', 'width', 'est_width', 'service', 'reversed', 'osm_id', 'access', 'maxspeed', 'oneway', 'ref', 'length', 'area', 'lanes']


# Remove node and edge attributes
network = remove_node_edge_attrs(
    network,
    node_attrs = remove_node_attr,
    edge_attrs = remove_edge_attr)


# Remove edges with specific highway types
remove_edge_attr = ['motorway', 'motorway_link', 'trunk', 'trunk_link']
network = remove_edge_by_attr_value(
    network,
    attr_name = 'highway',
    attr_values = remove_edge_attr,
    remove_isolated_nodes = True)

#### (3) Convert network to GeoDataFrame

In [ ]:
node_gdf, edge_gdf = graph_to_geodataframe(
    network,
    node_attr_x = 'proj_x',
    node_attr_y = 'proj_y',
    edge_attr_geom = 'geometry',
    crs = PROJECTED_CRS)

#### (4) Save network as GraphML

In [ ]:
from code_utils.network_process.osm_network_preprocess import convert_network_geometry_attr_to_wkt

# Convert network "geometry" attributes to WKT format
network = convert_network_geometry_attr_to_wkt(
    network, node_attr = None, edge_attr = 'geometry')

In [ ]:
# Save shapefile
node_gdf.to_file(PROJECT_PATH / 'shape_file/nodes.shp')
edge_gdf.to_file(PROJECT_PATH / 'shape_file/edges.shp')


# Save graph
nx.write_graphml(network,
    PROJECT_PATH / 'walking.graphml')